In [2]:
import numpy as np
import scipy as sc
import pandas as pd
import scipy.stats as stats
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# READ DATA
# read old protocol data
original_data_old = pd.read_csv('Protocol-99-H-0050B-data.csv')

# data.describe() # used to see count of data to cut the csv into the parts where there was data
old = original_data_old.head(59) # cut data to 59, which was count returned in data.describe()

# remove unnecessary white space at beginning and end of column headers
for col in old.columns:
    old = old.rename(columns={col:col.title().strip()}) # makes each word capitalized, removes leading and trailing spaces

old.columns = old.columns.str.replace('  ', ' ') # remove double spaces

# remove unnecessary white space at beginning and end of values
for row_idx in range(0, old.shape[0]):
    for col_idx in range(0, old.shape[1]):
        if old.dtypes[col_idx] == 'object':
            old.iloc[row_idx, col_idx] = str(old.iloc[row_idx, col_idx]).strip()
            old.iloc[row_idx, col_idx] = old.iloc[row_idx, col_idx].replace('  ', ' ')
            old.iloc[row_idx, col_idx] = old.iloc[row_idx, col_idx].title()
# print('Size: ', old.shape)
# n_old = old.shape[0]
# for column in old.columns:
#     print(column)
# old

In [4]:
# read new protocol data
original_data_new = pd.read_csv('Transplant_10H0154.csv')

# data.describe() # used to see count of data to cut the csv into the parts where there was data
new = original_data_new.head(35) # cut data to 35, which was count returned in data.describe()

# remove unnecessary white space at beginning and end of column headers
for col in new.columns:
    new = new.rename(columns={col:col.title().strip()}) # makes each word capitalized, removes leading and trailing spaces

new.columns = new.columns.str.replace('  ', ' ') # remove double spaces

# remove unnecessary white space at beginning and end of values
for row_idx in range(0, new.shape[0]):
    for col_idx in range(0, new.shape[1]):
        if new.dtypes[col_idx] == 'object':
            new.iloc[row_idx, col_idx] = str(new.iloc[row_idx, col_idx]).strip()
            new.iloc[row_idx, col_idx] = new.iloc[row_idx, col_idx].replace('  ', ' ')
            new.iloc[row_idx, col_idx] = new.iloc[row_idx, col_idx].title()
            
# print('Size: ', new.shape)
# n_new = new.shape[0]
# for column in new.columns:
#     print(column)
# new

In [5]:
# fisher exact
def fisher(row1, row2, oldcol, newcol, oldinput, newinput):
    """
    creates does a fisher exact test on the given data
    Inputs:
    - row1, row2: strings defining row names
    - oldcol, newcol: strings defining column names in original data
    Outputs:
    - oldinput, newinput: possible things that could be inputted into cells. Corresponds to row1.
    - p: the p-value"""
    df = pd.DataFrame({'Old': [0,0] , 'New': [0,0]})
    df.index = [row1, row2]
    for patient in old.index:
        if old.at[patient, oldcol] == oldinput:
            df.loc[row1, 'Old'] = df.loc[row1, 'Old'] + 1
        else:
            df.loc[row2, 'Old'] = df.loc[row2, 'Old'] + 1
    for patient in new.index:
        if new.at[patient, newcol] == newinput:
            df.loc[row1, 'New'] = df.loc[row1, 'New'] + 1
        else:
            df.loc[row2, 'New'] = df.loc[row2, 'New'] + 1

    print(df)

    df_p = stats.fisher_exact(df)
    return df_p[1]

In [6]:
# AGE wilcoxon rank test
ages = stats.mannwhitneyu(old['Ageonstudy'], new['Age At Consent'])
# age = stats.ttest_1samp(a=new['Age At Consent'], popmean=old['Ageonstudy'].mean())
np.round(ages.pvalue, 2)

0.73

In [7]:
# GENDER wilcoxon rank test
'''
    | Old(0) | New(1)
M(0)|
F(1)|
'''
gender=pd.DataFrame({'Old': [0,0] , 'New': [0,0]})
gender.index = ['M', 'F']

for patient in old.index:
    if old.at[patient, 'Sex'] == 'M':
        gender.loc['M', 'Old'] = gender.loc['M', 'Old'] + 1
    else:
        gender.loc['F', 'Old'] = gender.loc['F', 'Old'] + 1
for patient in new.index:
    if new.at[patient, 'Gender'] == 'M':
        gender.loc['M', 'New'] = gender.loc['M', 'New'] + 1
    else:
        gender.loc['F', 'New'] = gender.loc['F', 'New'] + 1

print(gender)

gender_p = stats.fisher_exact(gender)
np.round(gender_p[1], 2)

   Old  New
M   37   16
F   22   19


0.13

In [8]:
# PATIENT/DONOR GENDER MATCH
gender_match = pd.DataFrame({'Old': [0,0], 'New': [0,0]})
gender_match.index = ['Match', 'Mismatch']
for patient in old.index:
    if old.at[patient, 'Sex']==old.at[patient, 'Donorsex']:
        gender_match.at['Match', 'Old'] = gender_match.at['Match', 'Old'] + 1
    else: 
        gender_match.at['Mismatch', 'Old'] = gender_match.at['Mismatch', 'Old'] + 1
for patient in new.index:
    if new.at[patient, 'Gender']==new.at[patient, 'Donor Gender']:
        gender_match.at['Match', 'New'] = gender_match.at['Match', 'New'] + 1
    else:
        gender_match.at['Mismatch', 'New'] = gender_match.at['Mismatch', 'New'] + 1
print(gender_match)

test = pd.DataFrame({'New': [2, 13], 'Old': [17, 39]})
tst = stats.fisher_exact(gender_match)
tst

gender_match_p = stats.fisher_exact(gender_match)
np.round(gender_match_p[1], 2)

          Old  New
Match      29   18
Mismatch   30   17


1.0

In [9]:
# M DONOR/F PATIENT // F DONOR/M PATIENT
mf = pd.DataFrame({'Old': [0,0], 'New': [0,0]})
mf.index = ['M Donor F Patient', 'F Donor M Patient']
for patient in old.index:
    if old.at[patient, 'Sex']!=old.at[patient, 'Donorsex']:
        if old.at[patient, 'Sex']=='M':
            mf.at['F Donor M Patient', 'Old'] = mf.at['F Donor M Patient', 'Old'] + 1
        else:
            mf.at['M Donor F Patient', 'Old'] = mf.at['M Donor F Patient', 'Old'] + 1
for patient in new.index:
    if new.at[patient, 'Gender']!=new.at[patient, 'Donor Gender']:
        if new.at[patient, 'Gender']=='M':
            mf.at['F Donor M Patient', 'New'] = mf.at['F Donor M Patient', 'New'] + 1
        else:
            mf.at['M Donor F Patient', 'New'] = mf.at['M Donor F Patient', 'New'] + 1
print(mf)
mf_p = stats.fisher_exact(mf)
np.round(mf_p[1], 2)

                   Old  New
M Donor F Patient   11   10
F Donor M Patient   19    7


0.22

In [10]:
# HLA MATCH
hla = pd.DataFrame({'Old': [0,0], 'New': [0,0]})
hla.index = ['Match', 'Mismatch']
for patient in old.index:
    if old.at[patient, 'Hla']=='6/6':
        hla.at['Match', 'Old'] = hla.at['Match', 'Old'] + 1
    else:
        hla.at['Mismatch', 'Old'] = hla.at['Mismatch', 'Old'] + 1
for patient in new.index:
    if new.at[patient, 'Hla Match (Ou Of 10)']==10:
        hla.at['Match', 'New'] = hla.at['Match', 'New'] + 1
    else:
        hla.at['Mismatch', 'New'] = hla.at['Mismatch', 'New'] + 1
print(hla)
hla_p = stats.fisher_exact(hla)
np.round(hla_p[1], 2)

          Old  New
Match      53   34
Mismatch    6    1


0.25

In [11]:
# ALLOIMMUNIZATION
fisher('Alloimmunized', 'Not Alloimmunized', 'Alloalloimmunization (1=Yes)', 'Alloimmunized', 1, 'Yes')

                   Old  New
Alloimmunized       23   24
Not Alloimmunized   36   11


0.009971368141535514

In [12]:
# FERRITIN
ferritin_old = pd.Series(old['Ferritin'])
ferritin_new = pd.Series(new['Pre Transplant Ferritin (Mcg/L)'])
ferritin_old = ferritin_old.dropna()
ferritin_new = ferritin_new.dropna()

print(stats.mannwhitneyu(ferritin_old, ferritin_new))

MannwhitneyuResult(statistic=846.5, pvalue=0.22613938401992306)


In [13]:
# FERRITIN > 1000
temp = []
for patient in old.index:
    if old.at[patient, 'Ferritin'] > 1000:
        temp.append('Yes')
    else: temp.append('No')
old['Ferritin>1000'] = temp
temp = []
for patient in new.index:
    if new.at[patient, 'Pre Transplant Ferritin (Mcg/L)'] > 1000:
        temp.append('Yes')
    else: temp.append('No')
new['Ferritin>1000'] = temp
fisher('Ferritin<=1000', 'Ferritin>1000', 'Ferritin>1000', 'Ferritin>1000', 'No', 'No')

                Old  New
Ferritin<=1000   14    4
Ferritin>1000    45   31


0.1808075026710116

In [14]:
# TRANSPLANT CELL DOSAGE
# Cd34+ cells
cd34_p = stats.mannwhitneyu(old['Cd34 X10E6'], new['Total Cd34 Dose (X10 ^6 Cells /Kg)'])
print(cd34_p)
# CD3+ T cells
old['Adjusted CD3+'] = 100000000 * old['Cd3 X10E8']
new['Adjusted CD3+'] = 10000000 * new['Cd3 Cell-Back:Total Cd3 Dose (X10^7 Cells/Kg)']
cd3_p = stats.mannwhitneyu(old['Adjusted CD3+'], new['Adjusted CD3+'])
cd3_p

MannwhitneyuResult(statistic=569.0, pvalue=0.0002898983165919478)


MannwhitneyuResult(statistic=2065.0, pvalue=1.1496754049906183e-16)

In [15]:
# neutrophil recovery n (%)
df = pd.DataFrame({'Old': [0,0], 'New': [0,0]})
row1 = 'Neutrophil Recovery'
row2 = 'No Neutrophil Recovery'
s1 = pd.Series(old['Neutrophil_Recovery_Day'])
s2 = pd.Series(new['Days To Neutrophil Recovery'])
df.index = [row1, row2]
for patient in old.index:
    if not pd.isna(s1[patient]):
        df.loc[row1, 'Old'] = df.loc[row1, 'Old'] + 1
    else:
        df.loc[row2, 'Old'] = df.loc[row2, 'Old'] + 1
for patient in new.index:
    if not pd.isna(s2[patient]):
        df.loc[row1, 'New'] = df.loc[row1, 'New'] + 1
    else:
        df.loc[row2, 'New'] = df.loc[row2, 'New'] + 1

print(df)
print(stats.fisher_exact(df)[1])

                        Old  New
Neutrophil Recovery      58   35
No Neutrophil Recovery    1    0
1.0


In [16]:
# NEUTROPHIL RECOVERY median range
old_neutrophil = pd.Series(old['Neutrophil_Recovery_Day'])
new_neutrophil = pd.Series(new['Days To Neutrophil Recovery'])
# get rid of NaNs
old_max = old_neutrophil.max()
for patient in old_neutrophil.index:
    if pd.isna(old_neutrophil[patient]):
        old_neutrophil[patient] = old_max + 1
new_max = new_neutrophil.max()
for patient in new_neutrophil.index:
    if pd.isna(new_neutrophil[patient]):
        new_neutrophil[patient] = new_max + 1

neutrophil_p = stats.mannwhitneyu(old_neutrophil, new_neutrophil)
print(np.round(neutrophil_p.pvalue, 2))

0.85


In [17]:
# platelet recovery n (%)
df = pd.DataFrame({'Old': [0,0], 'New': [0,0]})
row1 = 'platelet Recovery'
row2 = 'No platelet Recovery'
s1 = pd.Series(old['Platelet_Recovery_Day'])
s2 = pd.Series(new['Days To Platelet Recovery Count Of 20'])
df.index = [row1, row2]
for patient in old.index:
    if not pd.isna(s1[patient]):
        df.loc[row1, 'Old'] = df.loc[row1, 'Old'] + 1
    else:
        df.loc[row2, 'Old'] = df.loc[row2, 'Old'] + 1
for patient in new.index:
    if not pd.isna(s2[patient]):
        df.loc[row1, 'New'] = df.loc[row1, 'New'] + 1
    else:
        df.loc[row2, 'New'] = df.loc[row2, 'New'] + 1

print(df)
print(stats.fisher_exact(df)[1])

                      Old  New
platelet Recovery      53   29
No platelet Recovery    6    6
0.3527340761193688


In [19]:
# PLATELET RECOVERY
old_platelet = pd.Series(old['Platelet_Recovery_Day'])
new_platelet = pd.Series(new['Days To Platelet Recovery Count Of 20'])
# get rid of NaNs
# old_platelet = old_platelet.dropna()
# new_platelet = new_platelet.dropna()
# old_max = old_platelet.max()
maximum = max(old_platelet.max(), new_platelet.max())
for patient in old_platelet.index:
    if pd.isna(old_platelet[patient]):
        old_platelet[patient] = maximum + 1
# new_max = new_platelet.max()
for patient in new_platelet.index:
    if pd.isna(new_platelet[patient]):
        new_platelet[patient] = maximum + 1
platelet_p = stats.mannwhitneyu(old_platelet, new_platelet)
print(platelet_p.pvalue)


2.784754487335916e-05


In [64]:
# t-cell chimerism n (%)
df = pd.DataFrame({'Old': [0,0], 'New': [0,0]})
row1 = 't-cell chimerism'
row2 = 'No t-cell chimerism'
s1 = pd.Series(old['Days_To_Full_Donor_T-Chim'])
s2 = pd.Series(new['Time (Days) To >= 95%Tcell T Cell Chimerism'])
df.index = [row1, row2]
for patient in old.index:
    if not pd.isna(s1[patient]):
        df.loc[row1, 'Old'] = df.loc[row1, 'Old'] + 1
    else:
        df.loc[row2, 'Old'] = df.loc[row2, 'Old'] + 1
for patient in new.index:
    if not pd.isna(s2[patient]):
        df.loc[row1, 'New'] = df.loc[row1, 'New'] + 1
    else:
        df.loc[row2, 'New'] = df.loc[row2, 'New'] + 1

print('t-cell chimerism')
print(df)
print(stats.fisher_exact(df)[1])

print()

# myeloid  chimerism n (%)
df = pd.DataFrame({'Old': [0,0], 'New': [0,0]})
row1 = 'myeloid chimerism'
row2 = 'No myeloid chimerism'
s1 = pd.Series(old['Days_To_Full_Donor_M-Chim'])
s2 = pd.Series(new['Time (Days) To >= 95% Myeloid Chimerism'])
df.index = [row1, row2]
for patient in old.index:
    if not pd.isna(s1[patient]):
        df.loc[row1, 'Old'] = df.loc[row1, 'Old'] + 1
    else:
        df.loc[row2, 'Old'] = df.loc[row2, 'Old'] + 1
for patient in new.index:
    if not pd.isna(s2[patient]):
        df.loc[row1, 'New'] = df.loc[row1, 'New'] + 1
    else:
        df.loc[row2, 'New'] = df.loc[row2, 'New'] + 1
print('myeloid chimerism')
print(df)
print(stats.fisher_exact(df)[1])



t-cell chimerism
                     Old  New
t-cell chimerism      58   32
No t-cell chimerism    1    3
0.14379893628498497

myeloid chimerism
                      Old  New
myeloid chimerism      58   34
No myeloid chimerism    1    1
1.0


In [16]:
# CHIMERISM
# T-cell chimerism
# get rid of nans
old_tchim = pd.Series(old['Days_To_Full_Donor_T-Chim'])
new_tchim = pd.Series(new['Time (Days) To >= 95%Tcell T Cell Chimerism'])
# old_max = old_tchim.max()
maximum = max(old_tchim.max(), new_tchim.max())
for patient in old_tchim.index:
    if pd.isna(old_tchim[patient]):
        old_tchim[patient] = maximum + 1
# new_max = new_tchim.max()
for patient in new_tchim.index:
    if pd.isna(new_tchim[patient]):
        new_tchim[patient] = maximum + 1
        
print(stats.mannwhitneyu(old['Days_To_Full_Donor_T-Chim'], new['Time (Days) To >= 95%Tcell T Cell Chimerism']))

# Myeloid chimerism
old_mchim = pd.Series(old['Days_To_Full_Donor_M-Chim'])
new_mchim = pd.Series(new['Time (Days) To >= 95% Myeloid Chimerism'])
# old_max = old_mchim.max()
maximum = max(old_mchim.max(), new_mchim.max())
for patient in old_mchim.index:
    if pd.isna(old_mchim[patient]):
        old_mchim[patient] = old_max + 1
# new_max = new_mchim.max()
for patient in new_mchim.index:
    if pd.isna(new_mchim[patient]):
        new_mchim[patient] = new_max + 1
        
print(stats.mannwhitneyu(old['Days_To_Full_Donor_M-Chim'], new['Time (Days) To >= 95% Myeloid Chimerism']))

MannwhitneyuResult(statistic=731.5, pvalue=0.015923600082173736)
MannwhitneyuResult(statistic=1560.5, pvalue=1.2013950722184115e-05)


In [17]:
# GVHD

# acute GVHD grade 2-4
acute2_4 = pd.DataFrame({'Old': [0, 0], 'New': [0, 0]})
acute2_4.index = ['Yes', 'No']

# acute GVHD grade 3-4
acute3_4 = pd.DataFrame({'Old': [0, 0], 'New': [0, 0]})
acute3_4.index = ['Yes', 'No']

for patient in old.index:
    if old.at[patient, 'Acute_Gvhd_Grade2To4 (0=Yes)'] == 0:
        acute2_4.at['Yes', 'Old'] = acute2_4.at['Yes', 'Old'] + 1
    else:
        acute2_4.at['No', 'Old'] = acute2_4.at['No', 'Old'] + 1
    if old.at[patient, 'Acute_Gvhd_Grade3To4 (0=Yes)'] == 0:
        acute3_4.at['Yes', 'Old'] = acute3_4.at['Yes', 'Old'] + 1
    else:
        acute3_4.at['No', 'Old'] = acute3_4.at['No', 'Old'] + 1
for patient in new.index:
    if new.at[patient, 'Acute Gvhd Grade'] >= 2:
        acute2_4.at['Yes', 'New'] = acute2_4.at['Yes', 'New'] + 1
    else:
        acute2_4.at['No', 'New'] = acute2_4.at['No', 'New'] + 1
    if new.at[patient, 'Acute Gvhd Grade'] >= 3:
        acute3_4.at['Yes', 'New'] = acute3_4.at['Yes', 'New'] + 1
    else:
        acute3_4.at['No', 'New'] = acute3_4.at['No', 'New'] + 1
print(acute2_4)
print(acute3_4)

# chronic GVHD
chronic = pd.DataFrame({'Old': [0, 0], 'New': [0, 0]})
chronic.index = ['Yes', 'No']
for patient in old.index:
    if old.at[patient, 'Chronic_Gvhd (1=Yes)'] == 1:
        chronic.at['Yes', 'Old'] = chronic.at['Yes', 'Old'] + 1
    else:
        chronic.at['No', 'Old'] = chronic.at['No', 'Old'] + 1
for patient in new.index:
    if new.at[patient, 'Chronic Gvhd (Yes/No)'] == 'Yes':
        chronic.at['Yes', 'New'] = chronic.at['Yes', 'New'] + 1
    else:
        chronic.at['No', 'New'] = chronic.at['No', 'New'] + 1
print(chronic)

acute2_4_p = stats.fisher_exact(acute2_4)
acute3_4_p = stats.fisher_exact(acute3_4)
chronic_p = stats.fisher_exact(chronic)
print('Acute Grade 2-4: ', np.round(acute2_4_p[1],5))
print('Acute Grade 3-4: ', np.round(acute3_4_p[1],5))
print('Chronic: ', chronic_p[1])

     Old  New
Yes   32    8
No    27   27
     Old  New
Yes   19    4
No    40   31
     Old  New
Yes   41    6
No    18   29
Acute Grade 2-4:  0.00468
Acute Grade 3-4:  0.02686
Chronic:  1.411245472415434e-06


In [18]:
# Extensive GVHD
fisher('Extensive GVHD', 'No Extensive GVHD', 'Cgvhd_Extensive', 'Cgvhd (Limited/Extensive)', 1, 'Extensive')

                   Old  New
Extensive GVHD      28    5
No Extensive GVHD   31   30


0.0015276971557956131

In [19]:
# CMV 

print(fisher('CMV at risk', 'CMV not at risk', 'Risk_Cmv', 'Cmv At Risk', 1, 'Yes'))

# reactivation
old['Cmv_Reactivation (0=Yes)']
new['Cmv Reactivation (Yes/No)']
print(fisher('CMV Reactivation', 'CMV no Reactivation', 'Cmv_Reactivation (0=Yes)', 'Cmv Reactivation (Yes/No)', 0, 'Yes'))

print()
print('Among at risk, number reactivated:')

cmvoutof = pd.DataFrame({'Old': [33, 53-33], 'New': [22, 29-22]})
cmvoutof.index = ['Reactivation', 'No Reactivation']
print(cmvoutof)
np.round(stats.fisher_exact(cmvoutof)[1],2)

                 Old  New
CMV at risk       53   29
CMV not at risk    6    6
0.3527340761193688
                     Old  New
CMV Reactivation      33   22
CMV no Reactivation   26   13
0.5258875630002572

Among at risk, number reactivated:
                 Old  New
Reactivation      33   22
No Reactivation   20    7


0.23

In [20]:
# cmv reactivation days
cmv_old = []
for patient in old.index:
    if old.at[patient, 'Cmv_Reactivation (0=Yes)'] == 0:
        cmv_old.append(old.at[patient, 'Cmvdays'])
cmv_new = []
for patient in new.index:
    if new.at[patient, 'Cmv Reactivation (Yes/No)'] == 'Yes':
        cmv_new.append(new.at[patient, 'Cmv Reactivation Days Post-Transplant'])
stats.mannwhitneyu(cmv_old, cmv_new)[1]

0.014993544590353299

In [21]:
# survival 200 day
survival = pd.DataFrame({'Old': [0,0], 'New': [0,0]})
survival.index = ['Dead at 200', 'Alive at 200']
for patient in old.index:
    if old.at[patient, 'Survival (0=Death)'] == 0 and old.at[patient, 'Survdays'] < 200:
        survival.at['Dead at 200', 'Old'] = survival.at['Dead at 200', 'Old'] + 1
    else:
        survival.at['Alive at 200', 'Old'] = survival.at['Alive at 200', 'Old'] + 1
for patient in new.index:
    if new.at[patient, 'Was Subject Alive At Day 200?'] == 'No':
        survival.at['Dead at 200', 'New'] = survival.at['Dead at 200', 'New'] + 1
    else:
        survival.at['Alive at 200', 'New'] = survival.at['Alive at 200', 'New'] + 1
np.round(stats.fisher_exact(survival)[1],2)

0.49

In [2]:
import math

# diagnosis
def _dfs(mat, pos, r_sum, c_sum, p_0, p):

    (xx, yy) = pos
    (r, c) = (len(r_sum), len(c_sum))

    mat_new = []

    for i in range(len(mat)):
        temp = []
        for j in range(len(mat[0])):
            temp.append(mat[i][j])
        mat_new.append(temp)

    if xx == -1 and yy == -1:
        for i in range(r-1):
            temp = r_sum[i]
            for j in range(c-1):
                temp -= mat_new[i][j]
            mat_new[i][c-1] = temp
        for j in range(c-1):
            temp = c_sum[j]
            for i in range(r-1):
                temp -= mat_new[i][j]
            mat_new[r-1][j] = temp
        temp = r_sum[r-1]
        for j in range(c-1):
            temp -= mat_new[r-1][j]
        if temp <0:
            return
        mat_new[r-1][c-1] = temp

        p_1 = 1
        for x in r_sum:
            p_1 *= math.factorial(x)
        for y in c_sum:
            p_1 *= math.factorial(y)

        n = 0
        for x in r_sum:
            n += x
        p_1 /= math.factorial(n)

        for i in range(len(mat_new)):
            for j in range(len(mat_new[0])):
                p_1 /= math.factorial(mat_new[i][j])
        if p_1 <= p_0 + 0.00000001:
            #print(mat_new)
            #print(p_1)
            p[0] += p_1
    else:
        max_1 = r_sum[xx]
        max_2 = c_sum[yy]
        for j in range(c):
            max_1 -= mat_new[xx][j]
        for i in range(r):
            max_2 -= mat_new[i][yy]
        for k in range(min(max_1,max_2)+1):
            mat_new[xx][yy] = k
            if xx == r-2 and yy == c-2:
                pos_new = (-1, -1)
            elif xx == r-2:
                pos_new = (0, yy+1)
            else:
                pos_new = (xx+1, yy)
            _dfs(mat_new, pos_new, r_sum, c_sum, p_0, p)


def fisher_exact(table):

    row_sum = []
    col_sum = []

    for i in range(len(table)):
        temp = 0
        for j in range(len(table[0])):
            temp += table[i][j]
        row_sum.append(temp)
    
    for j in range(len(table[0])):
        temp = 0
        for i in range(len(table)):
            temp += table[i][j]
        col_sum.append(temp)

    mat = [[0] * len(col_sum)] * len(row_sum)
    pos = (0, 0)

    p_0 = 1

    for x in row_sum:
        p_0 *= math.factorial(x)
    for y in col_sum:
        p_0 *= math.factorial(y)

    n = 0
    for x in row_sum:
        n += x
    p_0 /= math.factorial(n)

    for i in range(len(table)):
        for j in range(len(table[0])):
            p_0 /= math.factorial(table[i][j])

    p = [0]
    _dfs(mat, pos, row_sum, col_sum, p_0, p)

    return p[0]

# table [[21, 32], [10, 17], [4, 7], [0, 3]]
print(fisher_exact([[21, 32], 
                   [10, 17], 
                   [4, 7],
                   [0, 3]]))



0.7582032130518026
